In [2]:
from graph_extraction import GraphRAGExtractor, parse_fn
from graph_store import GraphRAGStore

from llama_index.llms.gemini import Gemini
from graph_extraction import KG_TRIPLET_EXTRACT_TMPL 
from llama_index.core import PropertyGraphIndex
from llama_index.core.schema import TextNode
llm = Gemini(
    model="models/gemini-2.0-flash",
    api_key="",  # uses GOOGLE_API_KEY env var by default
)

kg_extractor = GraphRAGExtractor(
    llm=llm,
    extract_prompt=KG_TRIPLET_EXTRACT_TMPL,
    max_paths_per_chunk=10,
    parse_fn=parse_fn,
)


/home/ngoctuhan/Documents/CallingPhoneSystemLLM/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Note: used to be `Neo4jPGStore`
graph_store = GraphRAGStore(
    llm,
    username="neo4j", 
    password="password", 
    url="neo4j://127.0.0.1:7687"
)

In [5]:
from llama_index.embeddings.openai import OpenAIEmbedding
embedding = OpenAIEmbedding(api_key="")

In [ ]:
# text_node1 = TextNode(text="Nguyễn Trãi (chữ Hán: 阮廌; 1380 – 19 tháng 9 năm 1442), hiệu là Ức Trai (抑齋), là một nhà chính trị, nhà văn, nhà văn hóa lớn của dân tộc Việt Nam. Ông đã tham gia tích cực cuộc Khởi nghĩa Lam Sơn do Lê Lợi lãnh đạo chống lại sự xâm lược của nhà Minh (Trung Quốc) với Đại Việt. Khi cuộc khởi nghĩa thành công vào năm 1428, Nguyễn Trãi trở thành một trong những khai quốc công thần của triều đại quân chủ nhà Hậu Lê trong Lịch sử Việt Nam.[2]")
# text_node2 = TextNode(text="""Nguyễn Du (chữ Hán: 阮攸; 3 tháng 1 năm 1766 – 16 tháng 9 năm 1820[1]) tên tự là Tố Như (素如), hiệu là Thanh Hiên (清軒), biệt hiệu là Hồng Sơn lạp hộ (鴻山獵戶), Nam Hải điếu đồ (南海釣屠), là một nhà thơ, nhà văn hóa lớn thời Lê mạt Nguyễn sơ ở Việt Nam. Ông được người Việt kính trọng tôn xưng là "Đại thi hào dân tộc"[2] và được UNESCO vinh danh là "Danh nhân văn hóa thế giới".[3]""")
# text_node3 = TextNode(text="""gRPC (acronym for gRPC Remote Procedure Calls[2]) is a cross-platform high-performance remote procedure call (RPC) framework. gRPC was initially created by Google, but is open source and is used in many organizations. Use cases range from microservices to the "last mile" of computing (mobile, web, and Internet of Things). gRPC uses HTTP/2 for transport, Protocol Buffers as the interface description language, and provides features such as authentication, bidirectional streaming and flow control, blocking or nonblocking bindings, and cancellation and timeouts. It generates cross-platform client and server bindings for many languages. Most common usage scenarios include connecting services in a microservices style architecture, or connecting mobile device clients to backend services.[3]""")
# text_node4 = TextNode(text="""Facebook is a social media and social networking service owned by the American technology conglomerate Meta. Created in 2004 by Mark Zuckerberg with four other Harvard College students and roommates, Eduardo Saverin, Andrew McCollum, Dustin Moskovitz, and Chris Hughes, its name derives from the face book directories often given to American university students. Membership was initially limited to Harvard students, gradually expanding to other North American universities. Since 2006, Facebook allows everyone to register from 13 years old, except in the case of a handful of nations, where the age requirement is 14 years.[6] As of December 2023, Facebook claimed almost 3.07 billion monthly active users worldwide.[7] As of November 2024, Facebook ranked as the third-most-visited website in the world, with 23% of its traffic coming from the United States.[8] It was the most downloaded mobile app of the 2010s.[9]""")
# nodes = [text_node1, text_node2, text_node3, text_node4]

nodes = [TextNode(text="""Nguyễn Trãi sinh năm 1380, mất năm 1442, hiệu là Ức Trai, quê ở làng Chi Ngại (Chi Linh, Hải Dương) sau rời về Nhị Khê (Thường Tín, Hà Tây, nay thuộc Hà Nội)
""")]

index = PropertyGraphIndex(
    nodes=nodes,
    llm=llm,
    kg_extractors=[kg_extractor],
    property_graph_store=graph_store,
    embed_model=embedding,
    show_progress=True,
)

Generating embeddings: 100%|██████████| 1/1 [00:01<00:00,  1.25s/it]


In [6]:
index_restore  = PropertyGraphIndex.from_existing(llm=llm,
    kg_extractors=[kg_extractor],
    property_graph_store=graph_store,
    embed_model=embedding,)

In [7]:
import re 
def get_entities(query_str, similarity_top_k):
        nodes_retrieved = index_restore.as_retriever(
            similarity_top_k=similarity_top_k
        ).retrieve(query_str)

        enitites = set()
        pattern = (
            r"^(\w+(?:\s+\w+)*)\s*->\s*([a-zA-Z\s]+?)\s*->\s*(\w+(?:\s+\w+)*)$"
        )

        for node in nodes_retrieved:
            matches = re.findall(
                pattern, node.text, re.MULTILINE | re.IGNORECASE
            )

            for match in matches:
                subject = match[0]
                obj = match[2]
                enitites.add(subject)
                enitites.add(obj)

        return list(enitites)

get_entities("Nguyễn Trãi mất năm nào?", 10)

['Nhị Khê',
 'Nhà Minh',
 'Nguyễn Du',
 'Hồng Sơn lạp hộ',
 '1442',
 'Nam Hải điếu đồ',
 'UNESCO',
 'Chi Linh',
 'Khởi Nghĩa Lam Sơn',
 'Chi Ngại',
 '1380',
 'Nhà Trần',
 'Trung Quốc',
 'Nguyễn Trãi',
 'Nhà Hậu Lê',
 'Ức Trai',
 'Việt Nam',
 'Thường Tín',
 'Hồ Quý Ly',
 'Đại thi hào dân tộc',
 'Tố Như',
 'Danh nhân văn hóa thế giới',
 'Thanh Hiên',
 'Nguyễn Phi Khanh']

In [16]:
index_restore.property_graph_store.build_communities()

In [17]:
index_restore.property_graph_store.get_community_summaries()

{0: 'Nguyễn Trãi, also known as Ức Trai, was born in either 1380 or Chi Ngại, which is located in Chi Linh (Hải Dương). He moved to Nhị Khê and died in 1442. He was the son of Nguyễn Phi Khanh and son-in-law of Trần Nguyên Đán. Nguyễn Trãi was a key figure in Vietnamese history, actively participating in the Lam Sơn Uprising and becoming a founding meritorious official of the Later Lê dynasty.',
 1: "Here's a summary of the relationships:\n\n*   **Nguyễn Phi Khanh and Nguyễn Trãi:** Nguyễn Phi Khanh is the father of Nguyễn Trãi.\n*   **Nguyễn Phi Khanh and Nhà Minh/Trung Quốc:** Nguyễn Phi Khanh surrendered to the Nhà Minh and subsequently went to Trung Quốc.\n*   **Nguyễn Trãi and Khởi Nghĩa Lam Sơn:** Nguyễn Trãi actively participated in the Lam Sơn Uprising.\n*   **Khởi Nghĩa Lam Sơn and Lê Lợi:** The Lam Sơn Uprising was led by Lê Lợi.\n*   **Khởi Nghĩa Lam Sơn and Nhà Minh:** The Lam Sơn Uprising was against the Ming dynasty's invasion.\n*   **Nhà Minh and Đại Việt:** The Ming dyn

In [18]:
entities = get_entities("Nguyễn Trãi là ai?", 10)
community_ids = []

for entity in entities:
    if entity in graph_store.entity_info:
        community_ids.extend(graph_store.entity_info[entity])
community_ids
# return list(set(community_ids))

[0,
 8,
 1,
 10,
 1,
 10,
 0,
 8,
 0,
 8,
 7,
 0,
 9,
 0,
 9,
 1,
 12,
 1,
 11,
 0,
 8,
 6,
 0,
 8,
 6,
 6,
 7]

In [19]:
community_summaries = index_restore.property_graph_store.get_community_summaries()
community_answers = [community_summary 
           for id, community_summary in community_summaries.items() if id in list(set(community_ids))
        ]
community_answers

['Nguyễn Trãi, also known as Ức Trai, was born in either 1380 or Chi Ngại, which is located in Chi Linh (Hải Dương). He moved to Nhị Khê and died in 1442. He was the son of Nguyễn Phi Khanh and son-in-law of Trần Nguyên Đán. Nguyễn Trãi was a key figure in Vietnamese history, actively participating in the Lam Sơn Uprising and becoming a founding meritorious official of the Later Lê dynasty.',
 "Here's a summary of the relationships:\n\n*   **Nguyễn Phi Khanh and Nguyễn Trãi:** Nguyễn Phi Khanh is the father of Nguyễn Trãi.\n*   **Nguyễn Phi Khanh and Nhà Minh/Trung Quốc:** Nguyễn Phi Khanh surrendered to the Nhà Minh and subsequently went to Trung Quốc.\n*   **Nguyễn Trãi and Khởi Nghĩa Lam Sơn:** Nguyễn Trãi actively participated in the Lam Sơn Uprising.\n*   **Khởi Nghĩa Lam Sơn and Lê Lợi:** The Lam Sơn Uprising was led by Lê Lợi.\n*   **Khởi Nghĩa Lam Sơn and Nhà Minh:** The Lam Sơn Uprising was against the Ming dynasty's invasion.\n*   **Nhà Minh and Đại Việt:** The Ming dynasty i

In [21]:
prompt = """
You are a helpful assistant that can answer questions base on information from the community.

Community:
{community_answers}

Question:
{"Nguyễn Trãi là ai?"}

Answer:
"""

llm.complete(prompt)


CompletionResponse(text='Nguyễn Trãi (1380-1442) là một nhà chính trị, nhà quân sự, nhà ngoại giao, nhà văn, nhà thơ Việt Nam thời Lê sơ. Ông là công thần khai quốc nhà Lê, có công lớn trong cuộc khởi nghĩa Lam Sơn chống quân Minh xâm lược. Nguyễn Trãi được công nhận là một trong những anh hùng dân tộc vĩ đại nhất và là một nhà văn hóa lớn của Việt Nam.\n', additional_kwargs={}, raw={'content': {'parts': [{'text': 'Nguyễn Trãi (1380-1442) là một nhà chính trị, nhà quân sự, nhà ngoại giao, nhà văn, nhà thơ Việt Nam thời Lê sơ. Ông là công thần khai quốc nhà Lê, có công lớn trong cuộc khởi nghĩa Lam Sơn chống quân Minh xâm lược. Nguyễn Trãi được công nhận là một trong những anh hùng dân tộc vĩ đại nhất và là một nhà văn hóa lớn của Việt Nam.\n'}], 'role': 'model'}, 'finish_reason': 1, 'avg_logprobs': -0.049460248744234124, 'safety_ratings': [], 'token_count': 0, 'grounding_attributions': [], 'block_reason': 0, 'usage_metadata': {'prompt_token_count': 33, 'candidates_token_count': 94, 'to

In [ ]:
llm.complete(prompt)
